#Prepare Environment

In [1]:
######################################
# Install Kafka 
######################################

# Ref: Tensorflow I/O official documentation, 2020-11-04, section: +++
# "Robust machine learning on streaming data using Kafka and Tensorflow-IO"
# https://www.tensorflow.org/io/tutorials/kafka#download_and_setup_kafka_and_zookeeper_instances

!curl -sSOL http://packages.confluent.io/archive/5.4/confluent-community-5.4.1-2.12.tar.gz
!tar -xzf confluent-community-5.4.1-2.12.tar.gz

# Start Kafka and Zookeeper Services
!cd confluent-5.4.1 && bin/zookeeper-server-start -daemon etc/kafka/zookeeper.properties
!cd confluent-5.4.1 && bin/kafka-server-start -daemon etc/kafka/server.properties
!cd confluent-5.4.1 && bin/schema-registry-start -daemon etc/schema-registry/schema-registry.properties
!sleep 10

!echo "kafka and zookeeper services are up and running.."
!confluent-5.4.1/bin/kafka-topics --version

# End of Ref. +++

kafka and zookeeper services are up and running..
5.4.1-ccs (Commit:1c8f62230319e789)


In [2]:
!ps -ef | grep kafka

root         309       1  0 21:35 ?        00:00:03 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -Djava.awt.headless=true -Xlog:gc*:file=/content/confluent-5.4.1/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=102400 -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/confluent-5.4.1/bin/../logs -Dlog4j.configuration=file:bin/../etc/kafka/log4j.properties -cp /content/confluent-5.4.1/bin/../share/java/kafka/*:/content/confluent-5.4.1/bin/../support-metrics-client/build/dependant-libs-2.12.10/*:/content/confluent-5.4.1/bin/../support-metrics-client/build/libs/*:/usr/share/java/support-metrics-client/* org.apache.zookeeper.server.quorum.QuorumPeerMain etc/kafka/zookeeper.properties
root         357       1  1 21:35 ?        00:00:19 java -Xmx1G -Xms1G -server -XX:+UseG1GC -XX:MaxGCPauseMi

In [3]:
# Create Kafka Topics

!confluent-5.4.1/bin/kafka-topics --create --zookeeper 127.0.0.1:2181 --replication-factor 1 --partitions 1 --topic train
!confluent-5.4.1/bin/kafka-topics --create --zookeeper 127.0.0.1:2181 --replication-factor 1 --partitions 1 --topic validation
!confluent-5.4.1/bin/kafka-topics --create --zookeeper 127.0.0.1:2181 --replication-factor 1 --partitions 1 --topic test

Error while executing topic command : Topic 'train' already exists.
[2023-02-21 21:57:11,899] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'train' already exists.
 (kafka.admin.TopicCommand$)
Error while executing topic command : Topic 'validation' already exists.
[2023-02-21 21:57:13,838] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'validation' already exists.
 (kafka.admin.TopicCommand$)
Error while executing topic command : Topic 'test' already exists.
[2023-02-21 21:57:15,748] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'test' already exists.
 (kafka.admin.TopicCommand$)


In [4]:
# Inspect Kafka Topics

!confluent-5.4.1/bin/kafka-topics --bootstrap-server 127.0.0.1:9092 --describe --topic train
!confluent-5.4.1/bin/kafka-topics --bootstrap-server 127.0.0.1:9092 --describe --topic validation
!confluent-5.4.1/bin/kafka-topics --bootstrap-server 127.0.0.1:9092 --describe --topic test

Topic: train	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: train	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: validation	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: validation	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: test	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: test	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


#1. Import thư viện

In [1]:
!pip install pyspark==3.3.1 
!pip install sparknlp
!pip install pyvi
!pip install unidecode
!pip install unicodedata
# Install kafka-python package
!pip install -q kafka-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata


In [2]:
import re
import unicodedata
import requests
import pandas as pd
import seaborn as sns

from pyvi import ViTokenizer

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lower, col, regexp_replace, count, mean, when, lit, create_map
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

from pyspark.ml import Pipeline
from pyspark.ml.feature import  Tokenizer, StopWordsRemover, HashingTF, IDF, CountVectorizer, NGram, Word2Vec
from pyspark.ml.linalg import Vectors, VectorUDT

from unidecode import unidecode
from itertools import chain
pd.options.display.max_rows = 9999

from pyspark.ml.feature import StringIndexer, VectorAssembler

from pyspark.ml.clustering import LDA
from pyspark.ml.classification import NaiveBayesModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col




#2. Khai báo Spark Session

In [3]:
spark = (
    SparkSession.builder.appName("ModelTraining")
    .config("spark.executor.memory", "16g")
    .getOrCreate()
)

#3. Read DATASET

###3.1 Read file data.csv (crawl từ web TGDD)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = spark.read.csv('/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA/data_comment.csv', header=True, inferSchema=True)

In [10]:
df.printSchema()

root
 |-- Id_comment: string (nullable = true)
 |-- Name_phone: string (nullable = true)
 |-- Name_comment: string (nullable = true)
 |-- Buy_place_comment: string (nullable = true)
 |-- Content_comment: string (nullable = true)
 |-- Used_comment: string (nullable = true)
 |-- label: integer (nullable = true)



In [11]:
df.show(4)

+----------+-----------------+---------------+-----------------+--------------------+--------------------+-----+
|Id_comment|       Name_phone|   Name_comment|Buy_place_comment|     Content_comment|        Used_comment|label|
+----------+-----------------+---------------+-----------------+--------------------+--------------------+-----+
|r-53650660|iPhone 14 Pro Max|Nguyễn Tiến Anh|  Đã mua tại TGDD|Dây sạc theo máy ...|Đã dùng khoảng 3 ...|    0|
|r-53646594|iPhone 14 Pro Max|           NHÂN|  Đã mua tại TGDD|Mua cho vợ xài ch...|Đã dùng khoảng 1 ...|    2|
|r-53642026|iPhone 14 Pro Max|           OANH|   Đã mua tại DMX|             Rất tốt|Đã dùng khoảng 3 ...|    1|
|r-53630446|iPhone 14 Pro Max|     Tuyết Ngân|  Đã mua tại TGDD|Sản phẩm rất tốt,...|Đã dùng khoảng 2 ...|    1|
+----------+-----------------+---------------+-----------------+--------------------+--------------------+-----+
only showing top 4 rows



In [12]:
print('Number of rows: \t', df.count())
print('Number of columns: \t', len(df.columns))

Number of rows: 	 3173
Number of columns: 	 7


Gán cột label:
*   positive: "1" (những comment đều khen về điện thoại)<br>
*   negative: "0" (những comment không tốt về điện thoại)<br>
*   neutral: "2" (vừa chê vừa khen hoặc những câu không nhằm mục đích đánh giá sản phẩm)


#4. Spark NLP Preprocessing

Tiền xử lý ngôn ngữ tự nhiên (NLP): Các bình luận người dùng được thu thập từ nhiều nguồn khác nhau, do đó đầu tiên cần loại bỏ các dữ liệu không liên quan hoặc không có giá trị, chẳng hạn như các đoạn văn bản quảng cáo, spam hoặc chữ ký. Sau đó, dữ liệu được tiền xử lý NLP bao gồm các bước như tách từ, xử lý từ sai chính tả và đồng nhất hóa từ viết tắt và từ viết hoa.<br>

Tiền xử lý dữ liệu văn bản tiếng Việt: Sau khi được tiền xử lý NLP, dữ liệu cần được xử lý thêm để loại bỏ các từ không có giá trị và chuẩn hóa dữ liệu. Các bước xử lý bao gồm:
<br>
Tách từ: Tiến hành tách từ bằng cách sử dụng các công cụ NLP như VnTokenizer hoặc pyvi để tách từ tiếng Việt.<br>
Loại bỏ stop words: Các stop words (từ dừng) là các từ rất phổ biến và không mang ý nghĩa như "và", "là", "từ", "đi", vv. Do đó, chúng cần được loại bỏ. Chuẩn hóa từ: Tiến hành chuẩn hóa từ bằng cách đưa các từ về dạng gốc (lemma) hoặc dạng đơn (stemming). Điều này giúp loại bỏ sự đa dạng từ vựng của ngôn ngữ tiếng Việt.<br>
Xử lý biểu cảm cảm xúc: Với dữ liệu bình luận, việc phân tích cảm xúc của người dùng là rất quan trọng. Có thể sử dụng các công cụ NLP như VnCoreNLP để phân tích cảm xúc hoặc xây dựng các mô hình máy học để phân loại bình luận theo cảm xúc.<br>
Vector hóa dữ liệu: Cuối cùng, dữ liệu được vector hóa để có thể sử dụng trong các mô hình máy học. Vector hóa dữ liệu là quá trình chuyển đổi dữ liệu văn bản thành các vector số để mô hình có thể xử lý được. Có nhiều phương pháp vector hóa<br>

###Download list stop words Vietnamese and combine and define functions

In [5]:
#Tải danh sách stop words tiếng việt từ github
stop_words_url_1 = 'https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt'
stop_words_data_1 = requests.get(stop_words_url_1).content.decode('utf-8')
stop_words_1 = stop_words_data_1.splitlines()

stop_words_url_2 = 'https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt'
stop_words_data_2 = requests.get(stop_words_url_2).content.decode('utf-8')
stop_words_2 = stop_words_data_2.splitlines()

In [6]:
# kết hợp các danh sách stop words có dấu
stop_words = list(set(stop_words_1 + stop_words_2))

In [7]:
# định nghĩa hàm để loại bỏ các stop words
def remove_stop_words(words):
    return [word for word in words if word not in stop_words]

# định nghĩa hàm để tách câu thành các từ và loại bỏ dấu
def tokenize(text):
    return ViTokenizer.tokenize(text).lower().split()

In [8]:
# đăng ký các hàm với pyspark
remove_stop_words_udf = udf(remove_stop_words, ArrayType(StringType()))
tokenize_udf = udf(tokenize, ArrayType(StringType()))

##4.1 Non-Accent Preprocessing

In [17]:
#lấy 3 cột trong df
df_non_accent = df.select('Id_comment','Content_comment', 'label')
df_non_accent.show()

+----------+--------------------+-----+
|Id_comment|     Content_comment|label|
+----------+--------------------+-----+
|r-53650660|Dây sạc theo máy ...|    0|
|r-53646594|Mua cho vợ xài ch...|    2|
|r-53642026|             Rất tốt|    1|
|r-53630446|Sản phẩm rất tốt,...|    1|
|r-53627015|Em muốn đổi từ ip...|    2|
|r-53625318|Rất tốt! Các bạn ...|    2|
|r-53622496|                  Ok|    1|
|r-53609586|1 sản phẩm đẹp gi...|    1|
|r-53606596|             Rất tốt|    1|
|r-53592098|hàng ngon nhân vi...|    1|
|r-53589531| sản phẩm chất lượng|    1|
|r-53585447|         Hàng OK lắm|    1|
|r-53582199|             Rất tốt|    1|
|r-53581205|Đẹp nhưng 13 prm ...|    0|
|r-53578243|Chất lượng dịch v...|    1|
|r-53577832|         Xài rất tốt|    1|
|r-53573116|nv thế giới dất n...|    2|
|r-53562547|Tgdd gần bến xe r...|    1|
|r-53561310|             Rất tốt|    1|
|r-53560815|                Hihi|    2|
+----------+--------------------+-----+
only showing top 20 rows



In [18]:
# Tạo hàm user-defined function để loại bỏ dấu
remove_diacritic = udf(lambda s: unidecode(s), StringType())

# Loại bỏ dấu trong cột text của DataFrame
df_non_accent = df_non_accent.withColumn("text_without_diacritic", remove_diacritic(col("Content_comment")))

In [19]:
df_non_accent.show()

+----------+--------------------+-----+----------------------+
|Id_comment|     Content_comment|label|text_without_diacritic|
+----------+--------------------+-----+----------------------+
|r-53650660|Dây sạc theo máy ...|    0|  Day sac theo may ...|
|r-53646594|Mua cho vợ xài ch...|    2|  Mua cho vo xai ch...|
|r-53642026|             Rất tốt|    1|               Rat tot|
|r-53630446|Sản phẩm rất tốt,...|    1|  San pham rat tot,...|
|r-53627015|Em muốn đổi từ ip...|    2|  Em muon doi tu ip...|
|r-53625318|Rất tốt! Các bạn ...|    2|  Rat tot! Cac ban ...|
|r-53622496|                  Ok|    1|                    Ok|
|r-53609586|1 sản phẩm đẹp gi...|    1|  1 san pham dep gi...|
|r-53606596|             Rất tốt|    1|               Rat tot|
|r-53592098|hàng ngon nhân vi...|    1|  hang ngon nhan vi...|
|r-53589531| sản phẩm chất lượng|    1|   san pham chat luong|
|r-53585447|         Hàng OK lắm|    1|           Hang OK lam|
|r-53582199|             Rất tốt|    1|               R

In [20]:
#===================loại bỏ các dòng không cần thiết và giá trị rỗng=======================#
df_non_accent = df_non_accent.filter((col("text_without_diacritic").isNotNull()) & (col("text_without_diacritic") != "") & (col("Content_comment").isNotNull()) & (col("Content_comment") != ""))
df_non_accent.show()

+----------+--------------------+-----+----------------------+
|Id_comment|     Content_comment|label|text_without_diacritic|
+----------+--------------------+-----+----------------------+
|r-53650660|Dây sạc theo máy ...|    0|  Day sac theo may ...|
|r-53646594|Mua cho vợ xài ch...|    2|  Mua cho vo xai ch...|
|r-53642026|             Rất tốt|    1|               Rat tot|
|r-53630446|Sản phẩm rất tốt,...|    1|  San pham rat tot,...|
|r-53627015|Em muốn đổi từ ip...|    2|  Em muon doi tu ip...|
|r-53625318|Rất tốt! Các bạn ...|    2|  Rat tot! Cac ban ...|
|r-53622496|                  Ok|    1|                    Ok|
|r-53609586|1 sản phẩm đẹp gi...|    1|  1 san pham dep gi...|
|r-53606596|             Rất tốt|    1|               Rat tot|
|r-53592098|hàng ngon nhân vi...|    1|  hang ngon nhan vi...|
|r-53589531| sản phẩm chất lượng|    1|   san pham chat luong|
|r-53585447|         Hàng OK lắm|    1|           Hang OK lam|
|r-53582199|             Rất tốt|    1|               R

In [21]:
#Tách từ
tokenizer = Tokenizer(inputCol="text_without_diacritic", outputCol="tokenized")

In [22]:
#loại bỏ stopwords trong tiếng việt
stop_words_no_accent = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('utf-8') for word in stop_words]
stop_words_remover = StopWordsRemover(inputCol="tokenized", outputCol="words", stopWords=stop_words_no_accent)

#chuyển đổi các câu văn bản thành các đặc trưng số (features) sử dụng thuật toán hashing trick.
hashing_tf = HashingTF(inputCol="words", outputCol="raw_features")

#đếm số lần xuất hiện của một từ trong tài liệu và tính toán IDF của từ đó trong tập dữ liệu đó. Đánh giá tầm quan trọng
idf = IDF(inputCol="raw_features", outputCol="features", minDocFreq=5)

In [23]:
# Khởi tạo pipeline
pipeline = Pipeline(stages=[tokenizer, stop_words_remover, hashing_tf, idf])

In [24]:
# Áp dụng pipeline lên dataframe
processed_df = pipeline.fit(df_non_accent).transform(df_non_accent)

In [25]:
# Chuẩn hóa cột label thành dạng số
label_indexer = StringIndexer(inputCol="label", outputCol="label_indexed").fit(processed_df)
processed_df = label_indexer.transform(processed_df)

In [26]:
# Chọn các cột cần sử dụng để huấn luyện mô hình
assembler = VectorAssembler(inputCols=["features"], outputCol="featured")
data = assembler.transform(processed_df).select("featured", "label_indexed")

In [27]:
# LDA topic modeling
lda = LDA(k=10, seed=123, optimizer="em", featuresCol="features")
model_lda = lda.fit(processed_df)

In [28]:
# Get topic distribution for each document
df_lda = model_lda.transform(processed_df).select(col("label_indexed"), col("features"))

In [29]:
df_lda.count()

3164

In [30]:
# Tạo vector đặc trưng từ các từ sử dụng Word2Vec
word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="words", outputCol="w2v_features")
w2v_model = word2Vec.fit(processed_df)
df_w2v = w2v_model.transform(processed_df)

In [31]:
assembler = VectorAssembler(inputCols=["features", "w2v_features"], outputCol="combined_features")
df_combined = assembler.transform(df_w2v).select("combined_features", "label_indexed")

In [32]:
df_combined.show()

+--------------------+-------------+
|   combined_features|label_indexed|
+--------------------+-------------+
|(262244,[19322,46...|          1.0|
|(262244,[15211,26...|          2.0|
|      (262244,[],[])|          0.0|
|(262244,[31577,46...|          0.0|
|(262244,[38819,46...|          2.0|
|(262244,[32899,49...|          2.0|
|(262244,[198197,2...|          0.0|
|(262244,[31577,38...|          0.0|
|      (262244,[],[])|          0.0|
|(262244,[5757,141...|          0.0|
|(262244,[31577,21...|          0.0|
|(262244,[149384,1...|          0.0|
|      (262244,[],[])|          0.0|
|(262244,[20743,35...|          1.0|
|(262244,[31331,16...|          0.0|
|(262244,[15211,26...|          0.0|
|(262244,[5757,101...|          2.0|
|(262244,[32899,57...|          0.0|
|      (262244,[],[])|          0.0|
|(262244,[262144,2...|          2.0|
+--------------------+-------------+
only showing top 20 rows



In [33]:
df_combined.count()

3164

In [34]:
# Lấy mẫu ngẫu nhiên với tỉ lệ 1/100
#sampled_data = df_combined.sample(False, 0.001)
#
# Đếm số lượng mẫu trong dữ liệu đã lấy mẫu
#sampled_data.count()

In [35]:
# Tách dữ liệu thành các tập train_data, dev_data, test_data
train_data, dev_data, test_data = df_combined.randomSplit([0.6, 0.2, 0.2], seed=42)

In [36]:
from pyspark.ml.classification import LogisticRegression

In [37]:
# Huấn luyện mô hình
LR = LogisticRegression(labelCol="label_indexed", featuresCol="combined_features")
pipeline = Pipeline(stages=[LR])
model = pipeline.fit(train_data)


In [38]:
# Đánh giá mô hình trên dev_data
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(dev_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

print("Accuracy_score_dev: {}".format(accuracy))
print("f1_score_dev: {}".format(f1))

Accuracy_score_dev: 0.7196850393700788
f1_score_dev: 0.7307149427636855


In [39]:
# Đánh giá mô hình trên test_data
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

print("Accuracy_score_test: {}".format(accuracy))
print("f1_score_test: {}".format(f1))

Accuracy_score_test: 0.7383820998278829
f1_score_test: 0.7437418918845766


##Accent Preprocessing

In [9]:
#lấy 3 cột trong df
df_accent = df.select('Id_comment','Content_comment', 'label')
df_accent.show()

+----------+--------------------+-----+
|Id_comment|     Content_comment|label|
+----------+--------------------+-----+
|r-53650660|Dây sạc theo máy ...|    0|
|r-53646594|Mua cho vợ xài ch...|    2|
|r-53642026|             Rất tốt|    1|
|r-53630446|Sản phẩm rất tốt,...|    1|
|r-53627015|Em muốn đổi từ ip...|    2|
|r-53625318|Rất tốt! Các bạn ...|    2|
|r-53622496|                  Ok|    1|
|r-53609586|1 sản phẩm đẹp gi...|    1|
|r-53606596|             Rất tốt|    1|
|r-53592098|hàng ngon nhân vi...|    1|
|r-53589531| sản phẩm chất lượng|    1|
|r-53585447|         Hàng OK lắm|    1|
|r-53582199|             Rất tốt|    1|
|r-53581205|Đẹp nhưng 13 prm ...|    0|
|r-53578243|Chất lượng dịch v...|    1|
|r-53577832|         Xài rất tốt|    1|
|r-53573116|nv thế giới dất n...|    2|
|r-53562547|Tgdd gần bến xe r...|    1|
|r-53561310|             Rất tốt|    1|
|r-53560815|                Hihi|    2|
+----------+--------------------+-----+
only showing top 20 rows



In [10]:
tokenizer = Tokenizer(inputCol="Content_comment", outputCol="tokenized")

stop_words_no_accent = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('utf-8') for word in stop_words]
stop_words_remover = StopWordsRemover(inputCol="tokenized", outputCol="words", stopWords=stop_words_no_accent)

hashing_tf = HashingTF(inputCol="words", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features", minDocFreq=5)

In [11]:
# Khởi tạo pipeline
pipeline = Pipeline(stages=[tokenizer, stop_words_remover, hashing_tf, idf])

In [12]:
# Áp dụng pipeline lên dataframe
processed_df = pipeline.fit(df_accent).transform(df_accent)

In [13]:
# Chuẩn hóa cột label thành dạng số
label_indexer = StringIndexer(inputCol="label", outputCol="label_indexed").fit(processed_df)
processed_df = label_indexer.transform(processed_df)

In [14]:
processed_df.show()

+----------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+-------------+
|Id_comment|     Content_comment|label|           tokenized|               words|        raw_features|            features|label_indexed|
+----------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+-------------+
|r-53650660|Dây sạc theo máy ...|    0|[dây, sạc, theo, ...|[dây, sạc, máy, b...|(262144,[14678,74...|(262144,[14678,74...|          1.0|
|r-53646594|Mua cho vợ xài ch...|    2|[mua, cho, vợ, xà...|[mua, vợ, xài, ch...|(262144,[8026,205...|(262144,[8026,205...|          2.0|
|r-53642026|             Rất tốt|    1|          [rất, tốt]|          [rất, tốt]|(262144,[8026,196...|(262144,[8026,196...|          0.0|
|r-53630446|Sản phẩm rất tốt,...|    1|[sản, phẩm, rất, ...|[sản, phẩm, rất, ...|(262144,[35726,46...|(262144,[35726,46...|          0.0|
|r-53627015|Em muốn đổi từ ip...| 

In [15]:
# Chọn các cột cần sử dụng để huấn luyện mô hình
assembler = VectorAssembler(inputCols=["features"], outputCol="featured")
data = assembler.transform(processed_df).select("featured", "label_indexed")

In [16]:
# LDA topic modeling
lda = LDA(k=10, seed=123, optimizer="em", featuresCol="features")
model_lda = lda.fit(processed_df)

In [17]:
# Get topic distribution for each document
df_lda = model_lda.transform(processed_df).select(col("label_indexed"), col("features"))

In [18]:
# Tạo vector đặc trưng từ các từ sử dụng Word2Vec
word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="words", outputCol="w2v_features")
w2v_model = word2Vec.fit(processed_df)
df_w2v = w2v_model.transform(processed_df)

In [19]:
assembler = VectorAssembler(inputCols=["features", "w2v_features"], outputCol="combined_features")
df_combined = assembler.transform(df_w2v).select("combined_features", "label_indexed")

In [20]:
# Chia dữ liệu thành train_data, dev_data, test_data
train_data, dev_data, test_data = data.randomSplit([0.6, 0.2, 0.2], seed=42)

In [21]:
# Huấn luyện mô hình
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="combined_features", labelCol="label_indexed")
pipeline = Pipeline(stages=[lr])
model = pipeline.fit(train_data)

In [24]:
# Đánh giá mô hình trên dev_data
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(dev_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

print("Accuracy_score_val: {}".format(accuracy))
print("f1_score_val: {}".format(f1))

Accuracy_score_val: 0.7720125786163522
f1_score_val: 0.7830638641841817


In [25]:
# Đánh giá mô hình trên test_data
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="label_indexed", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

print("Accuracy_score_test: {}".format(accuracy))
print("f1_score_test: {}".format(f1))

Accuracy_score_test: 0.7804459691252144
f1_score_test: 0.7845796641866235


In [ ]:
#result.write.csv("/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA (1)/result.csv", header=True)
#result.toPandas().to_csv("/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA (1)/result_accent.csv", index=False)

#Deploy on Streaming Data

##Kafka Producer

In [ ]:
######################################
# Stream tweets TO Kafka
######################################

%%writefile myKafkaProducer.py

from csv import reader
from time import sleep
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')

# (again) data source: 1.6 Million tweets, 240MB,
# https://www.kaggle.com/kazanova/sentiment140
csv_path = r'/content/drive/MyDrive/KhóaLuậnTốtNghiệp/BIGDATA/data_comment.csv'

with open(csv_path, 'r') as read_obj:
    for row in reader(read_obj):
        tweet = row[5]
        producer.send('tweets', key=None, value=bytes(tweet, 'utf-8'))
        sleep(0.1)


In [ ]:
######################################
# Start streaming 
######################################

!nohup python3 /content/myKafkaProducer.py &> kafka_producer.out &

In [ ]:
######################################
# Verify Kafka topics are receiving the tweets
######################################

print("Kafka topic event counts:\n")

!confluent-5.4.1/bin/kafka-run-class \
    kafka.tools.GetOffsetShell \
    --broker-list localhost:9092 --topic tweets \
    --time -1 ;


##Kafka Consumer

In [ ]:
######################################
# Predict Sentiment & LDA-Topic on Streaming Data
######################################

consumer = KafkaConsumer(
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='latest',
    enable_auto_commit=True,
)

consumer.subscribe(['tweets'])

i = 0
for message in consumer:

    tweet = message.value.decode()

    print(tweet)
    print('pred sentiment ->', predict_sentiment(tweet))
    print('pred topic     ->', predict_topic(tweet))
    print('\n')
    sleep(2)
    
    i += 1
    if i>5:
        break